In [1]:
import os
import json
import boto3
import psycopg2
from openai import OpenAI
from datetime import datetime
from dotenv import load_dotenv
from psycopg2.extras import Json

In [2]:
load_dotenv()

True

In [3]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
DB_HOST = 'localhost'
DB_PORT = "5435"
DB_NAME = "postgres"
DB_USER = "postgres"
DB_PASSWORD = "postgres"
S3_BUCKET_NAME = "xyz-support-images"
AWS_ACCESS_KEY_ID = "test"
AWS_SECRET_ACCESS_KEY = "test"
AWS_REGION = "us-east-1"
AWS_ENDPOINT_URL = "http://localhost:4566"

In [4]:
client = OpenAI(api_key=OPENAI_API_KEY)

In [5]:
s3_client = boto3.client(
    's3',
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    region_name=AWS_REGION,
    endpoint_url=AWS_ENDPOINT_URL,
)

In [6]:
with open("./data/support_ticket.json", "r") as f:
    support_tickets = json.load(f)

In [7]:
def create_database_tables():
    """Create necessary tables and extensions in the PostgreSQL database."""
    conn = None
    try:
        # Connect to PostgreSQL
        conn = psycopg2.connect(
            host=DB_HOST,
            port=DB_PORT,
            dbname=DB_NAME,
            user=DB_USER,
            password=DB_PASSWORD,
        )

        # Create a cursor
        cur = conn.cursor()

        # First try creating the extension - this will fail gracefully if it doesn't exist
        try:
            cur.execute("CREATE EXTENSION IF NOT EXISTS vector;")
            conn.commit()
            print("pgvector extension enabled successfully.")

            # Create ticket table with vector column if pgvector is available
            cur.execute(
                """
            CREATE TABLE IF NOT EXISTS support_tickets (
                id SERIAL PRIMARY KEY,
                ticket_id VARCHAR(20) UNIQUE NOT NULL,
                subject TEXT NOT NULL,
                description TEXT NOT NULL,
                customer JSONB NOT NULL,
                metadata JSONB NOT NULL,
                resolution JSONB,
                embedding vector(1536),
                created_at TIMESTAMP NOT NULL DEFAULT NOW()
            );
            """
            )
            conn.commit()
            print("Table with vector column created successfully.")
            return True

        except Exception as e:
            conn.rollback()
            print(f"Warning: Could not create pgvector extension: {e}")
            print("Creating table without vector column...")

            # Create table without vector column as fallback
            cur.execute(
                """
            CREATE TABLE IF NOT EXISTS support_tickets (
                id SERIAL PRIMARY KEY,
                ticket_id VARCHAR(20) UNIQUE NOT NULL,
                subject TEXT NOT NULL,
                description TEXT NOT NULL,
                customer JSONB NOT NULL,
                metadata JSONB NOT NULL,
                resolution JSONB,
                created_at TIMESTAMP NOT NULL DEFAULT NOW()
            );
            """
            )
            conn.commit()
            print("Table without vector column created successfully.")
            return False

    except Exception as e:
        print(f"Error creating database tables: {e}")
        if conn:
            conn.rollback()
        return False
    finally:
        if conn:
            conn.close()


def create_s3_bucket_if_not_exists():
    """Create the S3 bucket if it doesn't exist."""
    try:
        # Check if bucket exists
        s3_client.head_bucket(Bucket=S3_BUCKET_NAME)
        print(f"Bucket {S3_BUCKET_NAME} already exists.")
    except Exception:
        # Create the bucket
        try:
            # For LocalStack we don't need CreateBucketConfiguration
            s3_client.create_bucket(Bucket=S3_BUCKET_NAME)
            print(f"Created S3 bucket: {S3_BUCKET_NAME}")
        except Exception as e:
            print(f"Error creating S3 bucket: {e}")


# TODO: Add mock data if no image is present.
def upload_images_from_data_folder():
    """Upload images from the data folder to S3."""

    # First create the bucket
    try:
        create_s3_bucket_if_not_exists()
    except Exception as e:
        print(f"Error with S3 setup: {e}")
        return

    # Gather all image paths and descriptions from the tickets
    image_data = []
    for ticket in support_tickets:
        if "metadata" in ticket and "images" in ticket["metadata"]:
            ticket_id = ticket["ticket_id"]
            for image in ticket["metadata"]["images"]:
                image_data.append(
                    {
                        "s3_key": image["s3_key"],
                        "description": image["description"],
                        "ticket_id": ticket_id,
                    }
                )

    # Upload images from data folder to S3
    for img_info in image_data:
        try:
            # Construct the local file path
            local_file_path = os.path.join("data", img_info["s3_key"])

            # Check if the file exists
            if not os.path.exists(local_file_path):
                print(f"Warning: Local image {local_file_path} not found")
                continue

            # Read the file content
            with open(local_file_path, "rb") as file:
                file_content = file.read()

            # Upload to S3
            s3_client.put_object(
                Bucket=S3_BUCKET_NAME,
                Key=img_info["s3_key"],
                Body=file_content,
                ContentType="image/png",
            )
            print(
                f"Uploaded image from {local_file_path} to s3://{S3_BUCKET_NAME}/{img_info['s3_key']}"
            )
        except Exception as e:
            print(f"Error uploading image {img_info['s3_key']}: {e}")


def generate_embedding(text):
    """Generate embedding vector using OpenAI's embedding model."""
    try:
        # Updated API call for OpenAI client
        response = client.embeddings.create(model="text-embedding-ada-002", input=text)
        embedding = response.data[0].embedding
        return embedding
    except Exception as e:
        print(f"Error generating embedding: {e}")
        return None


def verify_s3_images(ticket):
    """Verify that images exist in S3 bucket and add presigned URLs."""
    if "metadata" in ticket and "images" in ticket["metadata"]:
        for image in ticket["metadata"]["images"]:
            s3_key = image["s3_key"]
            try:
                # Check if the image exists in S3
                s3_client.head_object(Bucket=S3_BUCKET_NAME, Key=s3_key)

                # Add a presigned URL (valid for 1 hour)
                presigned_url = s3_client.generate_presigned_url(
                    "get_object",
                    Params={"Bucket": S3_BUCKET_NAME, "Key": s3_key},
                    ExpiresIn=3600,
                )
                image["presigned_url"] = presigned_url
                image["exists"] = True

            except Exception as e:
                print(f"Warning: Image {s3_key} not found in S3: {e}")
                image["exists"] = False

    return ticket


def insert_ticket_data(ticket, has_vector=True):
    """Insert a support ticket with its embedding into the database."""
    conn = None
    try:
        # Verify S3 images
        ticket = verify_s3_images(ticket)

        # Connect to PostgreSQL
        conn = psycopg2.connect(
            host=DB_HOST,
            port=DB_PORT,
            dbname=DB_NAME,
            user=DB_USER,
            password=DB_PASSWORD,
        )

        # Create a cursor
        cur = conn.cursor()

        # Generate embedding if vector support is available
        embedding = None
        if has_vector:
            # Generate text for embedding
            embedding_text = f"{ticket['subject']} {ticket['description']}"
            if "resolution" in ticket and "solution" in ticket["resolution"]:
                embedding_text += f" {ticket['resolution']['solution']}"

            # Generate embedding
            embedding = generate_embedding(embedding_text)

        if has_vector and embedding:
            # Insert ticket data with embedding - cast to vector type
            cur.execute(
                """
            INSERT INTO support_tickets 
            (ticket_id, subject, description, customer, metadata, resolution, embedding, created_at)
            VALUES (%s, %s, %s, %s, %s, %s, %s::vector, %s)
            ON CONFLICT (ticket_id) 
            DO UPDATE SET
                subject = EXCLUDED.subject,
                description = EXCLUDED.description,
                customer = EXCLUDED.customer,
                metadata = EXCLUDED.metadata,
                resolution = EXCLUDED.resolution,
                embedding = EXCLUDED.embedding,
                created_at = EXCLUDED.created_at
            """,
                (
                    ticket["ticket_id"],
                    ticket["subject"],
                    ticket["description"],
                    Json(ticket["customer"]),
                    Json(ticket["metadata"]),
                    Json(ticket["resolution"]) if "resolution" in ticket else None,
                    str(embedding),  # Convert list to string for casting
                    datetime.now(),
                ),
            )
        else:
            # Insert ticket data without embedding
            cur.execute(
                """
            INSERT INTO support_tickets 
            (ticket_id, subject, description, customer, metadata, resolution, created_at)
            VALUES (%s, %s, %s, %s, %s, %s, %s)
            ON CONFLICT (ticket_id) 
            DO UPDATE SET
                subject = EXCLUDED.subject,
                description = EXCLUDED.description,
                customer = EXCLUDED.customer,
                metadata = EXCLUDED.metadata,
                resolution = EXCLUDED.resolution,
                created_at = EXCLUDED.created_at
            """,
                (
                    ticket["ticket_id"],
                    ticket["subject"],
                    ticket["description"],
                    Json(ticket["customer"]),
                    Json(ticket["metadata"]),
                    Json(ticket["resolution"]) if "resolution" in ticket else None,
                    datetime.now(),
                ),
            )

        # Commit the transaction
        conn.commit()
        print(f"Ticket {ticket['ticket_id']} inserted successfully.")
        return True

    except Exception as e:
        print(f"Error inserting ticket {ticket['ticket_id']}: {e}")
        if conn:
            conn.rollback()
        return False
    finally:
        if conn:
            conn.close()


def perform_vector_search(query_text, limit=5):
    """Perform a vector similarity search on the support tickets."""
    conn = None
    try:
        # Generate embedding for the query
        query_embedding = generate_embedding(query_text)

        if not query_embedding:
            return []

        # Connect to PostgreSQL
        conn = psycopg2.connect(
            host=DB_HOST,
            port=DB_PORT,
            dbname=DB_NAME,
            user=DB_USER,
            password=DB_PASSWORD,
        )

        # Create a cursor
        cur = conn.cursor()

        # Cast the embedding to the vector type explicitly
        # Use Euclidean distance (L2 norm)
        cur.execute(
            """
        SELECT 
            ticket_id, 
            subject, 
            description, 
            customer, 
            metadata, 
            resolution,
            embedding <-> %s::vector as distance
        FROM support_tickets
        ORDER BY embedding <-> %s::vector
        LIMIT %s;
        """,
            (str(query_embedding), str(query_embedding), limit),
        )

        # Fetch results
        results = []
        for row in cur.fetchall():
            results.append(
                {
                    "ticket_id": row[0],
                    "subject": row[1],
                    "description": row[2],
                    "customer": row[3],
                    "metadata": row[4],
                    "resolution": row[5],
                    "distance": row[6],
                }
            )

        return results

    except Exception as e:
        print(f"Error performing vector search: {e}")
        # Try an alternative query if the first one fails
        try:
            if conn:
                cur = conn.cursor()
                # Fall back to a non-vector search
                cur.execute(
                    """
                SELECT 
                    ticket_id, 
                    subject, 
                    description, 
                    customer, 
                    metadata, 
                    resolution
                FROM support_tickets
                WHERE 
                    subject ILIKE %s OR 
                    description ILIKE %s
                LIMIT %s;
                """,
                    (f"%{query_text}%", f"%{query_text}%", limit),
                )

                results = []
                for row in cur.fetchall():
                    results.append(
                        {
                            "ticket_id": row[0],
                            "subject": row[1],
                            "description": row[2],
                            "customer": row[3],
                            "metadata": row[4],
                            "resolution": row[5],
                            "distance": None,  # No distance in text search
                            "method": "text_fallback",
                        }
                    )

                print("Used text search fallback instead of vector search")
                return results
        except Exception as inner_e:
            print(f"Fallback search also failed: {inner_e}")
        return []
    finally:
        if conn:
            conn.close()


def debug_test_connection():
    """Test database and S3 connections."""
    print("\n--- Testing Database Connection ---")
    try:
        conn = psycopg2.connect(
            host=DB_HOST,
            port=DB_PORT,
            dbname=DB_NAME,
            user=DB_USER,
            password=DB_PASSWORD,
        )
        cur = conn.cursor()
        cur.execute("SELECT version();")
        version = cur.fetchone()
        print(f"Connected to PostgreSQL: {version[0]}")

        # Check extensions
        cur.execute(
            "SELECT name, installed_version FROM pg_available_extensions WHERE name = 'vector';"
        )
        ext = cur.fetchone()
        if ext:
            print(f"pgvector is available: {ext}")

            # Check if it's installed
            cur.execute(
                "SELECT extname, extversion FROM pg_extension WHERE extname = 'vector';"
            )
            installed = cur.fetchone()
            if installed:
                print(f"pgvector is installed: {installed}")
            else:
                print("pgvector is available but not installed yet")
        else:
            print("pgvector extension is not available")

        conn.close()
    except Exception as e:
        print(f"PostgreSQL connection error: {e}")

    print("\n--- Testing S3 Connection ---")
    try:
        # List S3 buckets
        response = s3_client.list_buckets()
        print(f"Connected to S3, found {len(response['Buckets'])} buckets")
        for bucket in response["Buckets"]:
            print(f" - {bucket['Name']}")
    except Exception as e:
        print(f"S3 connection error: {e}")

    print("\n--- End of Connection Tests ---\n")

In [8]:
def main():
    """Main function to process support tickets and store them in the database."""
    print("Starting support ticket data ingestion process...")

    # Debug connection test
    debug_test_connection()

    # Set up database tables - continue even if pgvector is not available
    has_vector = create_database_tables()

    # Upload images to S3
    print("\nUploading images to S3...")
    upload_images_from_data_folder()

    # Insert tickets
    for ticket in support_tickets:
        success = insert_ticket_data(ticket, has_vector)
        if not success:
            print(f"Failed to process ticket {ticket['ticket_id']}")

    # Only perform vector search if pgvector is available
    if has_vector:
        print("\nPerforming example vector search:")
        search_query = "login problems after password reset"
        results = perform_vector_search(search_query, limit=2)

        print(f"Search results for query: '{search_query}'")
        for i, result in enumerate(results):
            print(f"\nResult {i+1}:")
            if "distance" in result and result["distance"] is not None:
                print(f"Distance: {result['distance']:.4f}")
            if "method" in result and result["method"] == "text_fallback":
                print("(Using text search fallback)")
            print(f"Ticket ID: {result['ticket_id']}")
            print(f"Subject: {result['subject']}")
            print(
                f"Resolution: {result['resolution']['solution'] if result['resolution'] else 'N/A'}"
            )
    else:
        print("\nSkipping vector search (pgvector not available)")

In [9]:
main()

Starting support ticket data ingestion process...

--- Testing Database Connection ---
Connected to PostgreSQL: PostgreSQL 17.4 (Debian 17.4-1.pgdg120+2) on aarch64-unknown-linux-gnu, compiled by gcc (Debian 12.2.0-14) 12.2.0, 64-bit
pgvector is available: ('vector', None)
pgvector is available but not installed yet

--- Testing S3 Connection ---
Connected to S3, found 1 buckets
 - xyz-support-images

--- End of Connection Tests ---

pgvector extension enabled successfully.
Table with vector column created successfully.

Uploading images to S3...
Bucket xyz-support-images already exists.
Uploaded image from data/tickets/XYZ-2001/image.png to s3://xyz-support-images/tickets/XYZ-2001/image.png
Uploaded image from data/tickets/XYZ-2002/image.png to s3://xyz-support-images/tickets/XYZ-2002/image.png
Ticket XYZ-2001 inserted successfully.
Ticket XYZ-2002 inserted successfully.

Performing example vector search:
Search results for query: 'login problems after password reset'

Result 1:
Dista